In [2]:
from datasets import load_dataset
import random
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

2023-04-20 15:50:07.298547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Load the dataset
dataset = load_dataset("silicone", "maptask")

Found cached dataset silicone (/Users/moustaphalo/.cache/huggingface/datasets/silicone/maptask/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
# Separate the dataset into train val and test
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

# Extract the utterances and labels
train_X = train_dataset["Utterance"]
train_y = train_dataset["Label"]

val_X = val_dataset["Utterance"]
val_y = val_dataset["Label"]

test_X = test_dataset["Utterance"]
test_y = test_dataset["Label"]

In [31]:
# SWDA dataset
import pandas as pd
# Chemin du répertoire contenant les dossiers de données
chemin_dossier = "Data/swda"



fichier_train = os.path.join(chemin_dossier, "train.csv")
fichier_test = os.path.join(chemin_dossier, "test.csv")
train_dataset = pd.read_csv(fichier_train, delimiter='\t')
test_dataset = pd.read_csv(fichier_test, delimiter='\t')

# Extract the utterances and labels
train_X = train_dataset["Utterance"].tolist()
train_y = train_dataset["Dialogue_Act"]

test_X = test_dataset["Utterance"].tolist()
test_y = test_dataset["Dialogue_Act"]

# Map the labels to integers using a dictionary

# Create a dictionary to map the labels to integers
label2int = {}
for i, label in enumerate(set(train_y)):
    label2int[label] = i

# Map the labels to integers
train_y = [label2int[label] for label in train_y]
test_y = [label2int[label] for label in test_y]



In [39]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X)

In [40]:

# Convert text data to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_X)
test_sequences = tokenizer.texts_to_sequences(test_X)

In [41]:

# Pad sequences to have the same length
max_sequence_length = max(len(seq) for seq in train_sequences + test_sequences)
train_data_ = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_data_ = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

In [42]:
# Convert array to list
train_data = train_data_.tolist()
test_data = test_data_.tolist()

In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-04-19 21:58:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-19 21:58:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-19 21:58:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

In [43]:
# Load pre-trained GloVe embeddings
glove_path = os.path.join(
    os.path.expanduser("~"), "Documents/ANLP/Pro/glove.6B.100d.txt"
)  # Update with the path to your GloVe file
embedding_dim = 100  # The dimensionality of the GloVe embeddings
embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [44]:
# Create an embedding matrix for the words in the dataset
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < num_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [45]:
# Build the model
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(43, activation='softmax'))

2023-04-20 17:09:47.163396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 17:09:47.169005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 17:09:47.173213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [46]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_y, validation_data=(test_data, test_y), epochs=10, batch_size=32)

Epoch 1/10


2023-04-20 17:12:31.153420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 17:12:31.155724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 17:12:31.157686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6622/6623 [============================>.] - ETA: 0s - loss: 1.1768 - accuracy: 0.6395

2023-04-20 17:18:21.967372: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 17:18:21.972518: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 17:18:21.976169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6623/6623 [==============================] - 355s 53ms/step - loss: 1.1768 - accuracy: 0.6395 - val_loss: 1.0930 - val_accuracy: 0.6621
Epoch 2/10
6623/6623 [==============================] - 294s 44ms/step - loss: 1.0244 - accuracy: 0.6766 - val_loss: 1.0610 - val_accuracy: 0.6780
Epoch 3/10
6623/6623 [==============================] - 262s 40ms/step - loss: 0.9875 - accuracy: 0.6857 - val_loss: 1.0460 - val_accuracy: 0.6691
Epoch 4/10
6623/6623 [==============================] - 300s 45ms/step - loss: 0.9664 - accuracy: 0.6914 - val_loss: 1.0246 - val_accuracy: 0.6850
Epoch 5/10
6623/6623 [==============================] - 394s 59ms/step - loss: 0.9507 - accuracy: 0.6968 - val_loss: 1.0077 - val_accuracy: 0.6853
Epoch 6/10
6623/6623 [==============================] - 352s 53ms/step - loss: 0.9384 - accuracy: 0.6987 - val_loss: 1.0146 - val_accuracy: 0.6791
Epoch 7/10
6623/6623 [==============================] - 271s 41ms/step - loss: 0.9284 - accuracy: 0.7016 - val_loss: 1.0109 - val

In [47]:
# Evaluate the model
y_pred = model.predict(test_data)
y_pred_labels = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(test_y, y_pred_labels)
report = classification_report(test_y, y_pred_labels)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)















2023-04-20 18:07:20.279110: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 18:07:20.280380: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 18:07:20.283084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

85/85 [==============================] - 3s 20ms/step
Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           2       0.71      0.97      0.82       461
           3       0.00      0.00      0.00        11
           4       0.62      0.53      0.57       390
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         0
           7       0.67      0.81      0.74        48
           8       0.00      0.00      0.00         5
           9       1.00      0.29      0.44         7
          10       0.67      0.67      0.67         3
          11       1.00      0.50      0.67         2
          12       0.65      1.00      0.79        11
          13       1.00      0.33      0.50         3
          14       0.00      0.00      0.00        11
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00

/Users/moustaphalo/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/moustaphalo/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/moustaphalo/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mo